# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-23-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-23-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-24 04:21:55,33.93911,67.709953,107957,4366,66102.0,37489.0,Afghanistan,277.322375,4.044203
1,NaN,NaN,NaN,Albania,2021-06-24 04:21:55,41.15330,20.168300,132497,2455,129955.0,87.0,Albania,4604.107304,1.852872
2,NaN,NaN,NaN,Algeria,2021-06-24 04:21:55,28.03390,1.659600,137049,3660,95331.0,38058.0,Algeria,312.533045,2.670578
3,NaN,NaN,NaN,Andorra,2021-06-24 04:21:55,42.50630,1.521800,13873,127,13688.0,58.0,Andorra,17955.089627,0.915447
4,NaN,NaN,NaN,Angola,2021-06-24 04:21:55,-11.20270,17.873900,38002,878,32322.0,4802.0,Angola,115.626149,2.310405


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,91458,93272,93288,96531,98734,98734,98734,103902,105749,107957
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132461,132469,132476,132481,132484,132488,132490,132490,132496,132497
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,133742,134115,134458,134840,135219,135586,135821,136294,136679,137049


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3612,3683,3683,3842,3934,3934,3934,4215,4293,4366
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2453,2454,2454,2454,2454,2454,2454,2454,2455,2455
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3579,3588,3598,3605,3615,3624,3631,3641,3650,3660


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,61706,61820,62397,62698,63426,63875,64401,65071,65565,66102
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129865,129879,129888,129895,129903,129910,129918,129918,129941,129955
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,93094,93355,93586,93831,94093,94336,94571,94822,95084,95331


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7229,7230,7230,7230,7241,7241,7241,7242,7242,7244
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21805,21833,21839,21839,21868,21868,21868,21901,21901,21921
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2345,2346,2346,2346,2345,2345,2345,2345,2345,2344


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,313,313,313,313,314,314,314,314,314,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,59,59,59,59,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
655,1001.0,Autauga,Alabama,US,2021-06-24 04:21:55,32.539527,-86.644082,7244,113,NaN,NaN,"Autauga, Alabama, US",12966.045571,1.559912
692,1075.0,Lamar,Alabama,US,2021-06-24 04:21:55,33.779950,-88.096680,1472,37,NaN,NaN,"Lamar, Alabama, US",10662.803332,2.513587
693,1077.0,Lauderdale,Alabama,US,2021-06-24 04:21:55,34.901719,-87.656247,9621,250,NaN,NaN,"Lauderdale, Alabama, US",10375.394968,2.598482


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33577651,602837,0.0
India,30028709,390660,28994855.0
Brazil,18169881,507109,16008271.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33577651,602837,0.0,32974814.0,2021-06-24 04:21:55,37.950547,-91.419547
India,30028709,390660,28994855.0,643194.0,2021-06-24 04:21:55,23.088275,81.806127
Brazil,18169881,507109,16008271.0,1654501.0,2021-06-24 04:21:55,-12.669522,-48.480493
France,5824127,111024,403326.0,5309777.0,2021-06-24 04:21:55,6.430808,-34.730285
Turkey,5387545,49358,5248862.0,89325.0,2021-06-24 04:21:55,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3811723,63424,0.0
Texas,2989604,52183,0.0
Florida,2354416,37555,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3811723,63424,0.0,3748299.0,2021-06-24 04:21:55,37.843962,-120.728594
Texas,2989604,52183,0.0,2937421.0,2021-06-24 04:21:55,31.660643,-98.653069
Florida,2354416,37555,0.0,2316861.0,2021-06-24 04:21:55,28.940755,-82.700744
New York,2112649,53639,0.0,2059010.0,2021-06-24 04:21:55,42.544151,-75.474183
Illinois,1389892,25606,0.0,1364286.0,2021-06-24 04:21:55,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1248161,24461,0.0
Arizona,Maricopa,557651,10238,0.0
Illinois,Cook,556556,10470,0.0
Florida,Miami-Dade,506428,6472,0.0
Texas,Harris,403187,6557,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1248161,24461,0.0,1223700.0,2021-06-24 04:21:55,34.308284,-118.228241,6037.0
Arizona,Maricopa,557651,10238,0.0,547413.0,2021-06-24 04:21:55,33.348359,-112.491815,4013.0
Illinois,Cook,556556,10470,0.0,546086.0,2021-06-24 04:21:55,41.841448,-87.816588,17031.0
Florida,Miami-Dade,506428,6472,0.0,499956.0,2021-06-24 04:21:55,25.611236,-80.551706,12086.0
Texas,Harris,403187,6557,0.0,396630.0,2021-06-24 04:21:55,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,103902,132490,136294,13864,37748,1263,4277395,224253,30366,649670,...,4656536,356382,106452,4,262038,13530,312811,6889,130631,42195
2021-06-22,105749,132496,136679,13864,37874,1263,4298782,224330,30380,649728,...,4668043,358461,106847,4,263372,13782,313015,6892,133659,42714
2021-06-23,107957,132497,137049,13873,38002,1263,4326101,224430,30404,649845,...,4683986,360247,107266,4,263372,13989,313015,6898,137026,43480


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,4215,2454,3641,127,868,42,89490,4499,910,10680,...,128245,5316,718,1,2973,69,3554,1355,1691,1685
2021-06-22,4293,2455,3650,127,875,42,90281,4502,910,10684,...,128272,5350,720,1,2989,69,3555,1355,1744,1691
2021-06-23,4366,2455,3660,127,878,42,90986,4503,910,10686,...,128291,5374,722,1,2989,70,3555,1355,1794,1692


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,65071,129918,94822,13665,31782,1221,3910835,216112,23747,636365,...,15612,326871,102163,3,242357,5453,306312,3958,110450,37200
2021-06-22,65565,129941,95084,13665,32003,1221,3928389,216182,23750,636523,...,15616,329662,102546,3,243840,5546,306532,3980,111844,37288
2021-06-23,66102,129955,95331,13688,32322,1221,3944323,216230,23752,636740,...,15705,332112,103074,3,243840,5684,306532,3990,113109,37477


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7229,7230,7230,7230,7241,7241,7241,7242,7242,7244
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1462,1462,1465,1465,1468,1468,1468,1469,1469,1472
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9591,9592,9596,9596,9603,9603,9603,9612,9612,9621
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3124,3125,3129,3129,3133,3133,3133,3134,3134,3135
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16248,16259,16265,16265,16278,16278,16278,16287,16287,16301


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,549013,70963,0,890111,345695,3809710,554090,348757,109564,49,...,866324,2986414,411770,24368,3772,678765,447724,163382,676820,61630
2021-06-22,549013,70963,0,890533,346180,3810808,554473,348802,109577,49,...,866419,2988388,412019,24371,3772,678909,448142,163627,676931,61704
2021-06-23,549394,71035,0,891023,346569,3811723,554977,348840,109599,49,...,866608,2989604,412546,24379,3772,679137,448945,163689,677048,61776


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-21        7242   21901    2345  2685   6950    1245   2246   14724   
2021-06-22        7242   21901    2345  2685   6950    1245   2246   14724   
2021-06-23        7244   21921    2344  2686   6957    1246   2250   14725   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-21         3728     1873  ...    2894    677     3250      802   
2021-06-22         3728     1873  ...    2898    677     3251      802   
2021-06-23         3731     1875  ...    2898    677     3259      799   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-21           4696  3798  2305          0      927    656  
2021-06-22           4699  3798  2308          0      927    656  
2021-06-23           4712  3798  2304          0      927    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,11311,373,0,17843,5876,63356,6736,8269,1683,0,...,12524,52118,2330,256,30,11360,5838,2863,8064,734
2021-06-22,11311,373,0,17858,5884,63373,6740,8271,1691,0,...,12534,52150,2333,256,30,11367,5843,2870,8067,734
2021-06-23,11328,374,0,17868,5887,63424,6754,8271,1693,0,...,12539,52183,2336,256,30,11368,5889,2872,8077,740


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-21         113     314      59   64    139      42     71     327   
2021-06-22         113     314      59   64    139      42     71     327   
2021-06-23         113     314      60   64    139      42     71     329   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-21          124       45  ...      30     11       31        7   
2021-06-22          124       45  ...      30     11       31        7   
2021-06-23          124       45  ...      30     12       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-21             40    11    13          0       26      6  
2021-06-22             40    11    13          0       26      6  
2021-06-23             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.052343,0.000000,0.003483,0.001589,0.001858,0.0,0.002016,0.000116,0.000362,0.000145,...,0.002253,0.004275,0.004027,0.0,0.004978,0.020516,0.000441,0.000581,0.012384,0.009957
2021-06-22,0.017776,0.000045,0.002825,0.000000,0.003338,0.0,0.005000,0.000343,0.000461,0.000089,...,0.002471,0.005834,0.003711,0.0,0.005091,0.018625,0.000652,0.000435,0.023180,0.012300
2021-06-23,0.020880,0.000008,0.002707,0.000649,0.003380,0.0,0.006355,0.000446,0.000790,0.000180,...,0.003415,0.004982,0.003921,0.0,0.000000,0.015020,0.000000,0.000871,0.025191,0.017933


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.071429,0.000000,0.002754,0.0,0.010477,0.0,0.005020,0.000222,0.0,0.000000,...,0.000039,0.008537,0.000000,0.0,0.005071,0.045455,0.000845,0.0,0.028589,0.007775
2021-06-22,0.018505,0.000407,0.002472,0.0,0.008065,0.0,0.008839,0.000667,0.0,0.000375,...,0.000211,0.006396,0.002786,0.0,0.005382,0.000000,0.000281,0.0,0.031342,0.003561
2021-06-23,0.017004,0.000000,0.002740,0.0,0.003429,0.0,0.007809,0.000222,0.0,0.000187,...,0.000148,0.004486,0.002778,0.0,0.000000,0.014493,0.000000,0.0,0.028670,0.000591


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.010404,0.000000,0.002654,0.001099,0.003346,0.0,0.005301,0.000292,0.000211,0.000377,...,0.000128,0.010439,0.004632,0.0,0.005910,0.042838,0.000235,0.001265,0.013675,0.000430
2021-06-22,0.007592,0.000177,0.002763,0.000000,0.006954,0.0,0.004489,0.000324,0.000126,0.000248,...,0.000256,0.008539,0.003749,0.0,0.006119,0.017055,0.000718,0.005558,0.012621,0.002366
2021-06-23,0.008190,0.000108,0.002598,0.001683,0.009968,0.0,0.004056,0.000222,0.000084,0.000341,...,0.005699,0.007432,0.005149,0.0,0.000000,0.024883,0.000000,0.002513,0.011310,0.005069


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,0.000649,0.001242,NaN,0.000432,0.000260,0.000381,0.000401,0.000264,0.000146,0.0,...,0.000343,0.000253,0.000389,0.000328,0.00533,0.000171,0.001165,0.000000,0.000217,0.002277
2021-06-22,0.000000,0.000000,NaN,0.000474,0.001403,0.000288,0.000691,0.000129,0.000119,0.0,...,0.000110,0.000661,0.000605,0.000123,0.00000,0.000212,0.000934,0.001500,0.000164,0.001201
2021-06-23,0.000694,0.001015,NaN,0.000550,0.001124,0.000240,0.000909,0.000109,0.000201,0.0,...,0.000218,0.000407,0.001279,0.000328,0.00000,0.000336,0.001792,0.000379,0.000173,0.001167


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-21      0.000138  0.001509  0.000000  0.000000  0.000720  0.001609   
2021-06-22      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-06-23      0.000276  0.000913 -0.000426  0.000372  0.001007  0.000803   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-21      0.001784  0.000340  0.000000  0.000000  ...  0.002772   
2021-06-22      0.000000  0.000000  0.000000  0.000000  ...  0.001382   
2021-06-23      0.001781  0.000068  0.000805  0.001068  ...  0.000000   

Province_State                                                              \
Admin2           Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-21      0.00744  0.000000  0.000000   0.004062  0.000263  0.001303   
2021-06-22      0.00000  0.000308  0.000000   0.000639  0.000000  0.001302   
2021-06-23      0.00000  0.002461 -0.003741   0.002767  0.000000 -0.001733   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-06-21            NaN  0.002162    0.0  
2021-06-22            NaN  0.000000    0.0  
2021-06-23            NaN  0.000000    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,0.000442,0.000000,NaN,0.000000,0.000340,0.000332,0.000743,-0.000121,0.000000,NaN,...,0.000639,0.000154,0.000000,0.0,0.0,0.000793,0.003093,0.000000,0.002113,0.000000
2021-06-22,0.000000,0.000000,NaN,0.000841,0.001361,0.000268,0.000594,0.000242,0.004753,NaN,...,0.000798,0.000614,0.001288,0.0,0.0,0.000616,0.000856,0.002445,0.000372,0.000000
2021-06-23,0.001503,0.002681,NaN,0.000560,0.000510,0.000805,0.002077,0.000000,0.001183,NaN,...,0.000399,0.000633,0.001286,0.0,0.0,0.000088,0.007873,0.000697,0.001240,0.008174


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga Baldwin   Barbour Bibb Blount Bullock Butler   Calhoun   
2021-06-21         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.000000   
2021-06-22         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.000000   
2021-06-23         0.0     0.0  0.016949  0.0    0.0     0.0    0.0  0.006116   

Province_State                    ... Wyoming                              \
Admin2         Chambers Cherokee  ...    Park    Platte Sheridan Sublette   
2021-06-21          0.0      0.0  ...     0.0  0.000000      0.0      0.0   
2021-06-22          0.0      0.0  ...     0.0  0.000000      0.0      0.0   
2021-06-23          0.0      0.0  ...     0.0  0.090909      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-21            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-22            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-23            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.028968,0.000012,0.002860,0.000830,0.002464,4.752241e-09,0.002743,0.000219,0.000465,0.000177,...,0.002157,0.004835,0.003899,7.228014e-20,0.005027,0.024314,0.000390,0.000693,0.015150,0.007844
2021-06-22,0.023372,0.000029,0.002843,0.000415,0.002901,2.376120e-09,0.003871,0.000281,0.000463,0.000133,...,0.002314,0.005335,0.003805,3.614007e-20,0.005059,0.021470,0.000521,0.000564,0.019165,0.010072
2021-06-23,0.022126,0.000018,0.002775,0.000532,0.003140,1.188060e-09,0.005113,0.000363,0.000627,0.000157,...,0.002865,0.005158,0.003863,1.807004e-20,0.002530,0.018245,0.000261,0.000718,0.022178,0.014003


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.038878,0.000003,0.002484,7.343482e-18,0.007048,4.082391e-12,0.004795,0.000365,4.659147e-25,0.000084,...,0.000057,0.007623,0.000680,0.0,0.005418,0.035933,0.000652,0.000531,0.027957,0.006067
2021-06-22,0.028692,0.000205,0.002478,3.671741e-18,0.007556,2.041196e-12,0.006817,0.000516,2.329574e-25,0.000229,...,0.000134,0.007009,0.001733,0.0,0.005400,0.017966,0.000467,0.000266,0.029650,0.004814
2021-06-23,0.022848,0.000103,0.002609,1.835871e-18,0.005492,1.020598e-12,0.007313,0.000369,1.164787e-25,0.000208,...,0.000141,0.005748,0.002255,0.0,0.002700,0.016230,0.000233,0.000133,0.029160,0.002703


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-21,0.009227,0.000031,0.002610,0.000633,0.003724,0.000180,0.005684,0.000368,0.000264,0.000418,...,0.000201,0.010378,0.004052,1.734723e-18,0.005797,0.041140,0.000529,0.007955,0.018336,0.000515
2021-06-22,0.008409,0.000104,0.002686,0.000316,0.005339,0.000090,0.005086,0.000346,0.000195,0.000333,...,0.000229,0.009458,0.003900,8.673617e-19,0.005958,0.029097,0.000624,0.006757,0.015479,0.001440
2021-06-23,0.008300,0.000106,0.002642,0.001000,0.007653,0.000045,0.004571,0.000284,0.000140,0.000337,...,0.002964,0.008445,0.004525,4.336809e-19,0.002979,0.026990,0.000312,0.004635,0.013395,0.003254


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,0.000376,0.000729,NaN,0.000466,0.000417,0.000288,0.000522,0.000152,0.000187,2.858671e-140,...,0.000200,0.000269,0.000536,0.000194,0.003002,0.000149,0.000871,0.000031,0.000129,0.001270
2021-06-22,0.000188,0.000365,NaN,0.000470,0.000910,0.000288,0.000606,0.000141,0.000153,1.429335e-140,...,0.000155,0.000465,0.000570,0.000159,0.001501,0.000181,0.000902,0.000765,0.000146,0.001236
2021-06-23,0.000441,0.000690,NaN,0.000510,0.001017,0.000264,0.000758,0.000125,0.000177,7.146677e-141,...,0.000186,0.000436,0.000925,0.000243,0.000751,0.000258,0.001347,0.000572,0.000160,0.001201


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-21      0.000173  0.000858 -0.000023  0.000057  0.000391  0.000855   
2021-06-22      0.000087  0.000429 -0.000012  0.000029  0.000195  0.000428   
2021-06-23      0.000181  0.000671 -0.000219  0.000201  0.000601  0.000615   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-21      0.000966  0.000185  0.000053  0.000034  ...  0.001430   
2021-06-22      0.000483  0.000093  0.000026  0.000017  ...  0.001406   
2021-06-23      0.001132  0.000080  0.000416  0.000542  ...  0.000703   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-21      0.003711  0.000026  0.000006   0.002206  0.000135  0.000675   
2021-06-22      0.001856  0.000167  0.000003   0.001422  0.000068  0.000988   
2021-06-23      0.000928  0.001314 -0.001869   0.002095  0.000034 -0.000373   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-21           -1.0  0.001089  9.396998e-08  
2021-06-22           -1.0  0.000545  4.698499e-08  
2021-06-23           -1.0  0.000272  2.349249e-08  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-21,0.000339,0.000005,NaN,0.000277,0.000239,0.000379,0.000650,-0.000021,0.000372,NaN,...,0.000376,0.000261,0.000385,1.496006e-08,0.000818,0.000651,0.001708,0.000064,0.001128,0.000097
2021-06-22,0.000170,0.000003,NaN,0.000559,0.000800,0.000323,0.000622,0.000110,0.002563,NaN,...,0.000587,0.000437,0.000836,7.480030e-09,0.000409,0.000634,0.001282,0.001255,0.000750,0.000049
2021-06-23,0.000836,0.001342,NaN,0.000559,0.000655,0.000564,0.001350,0.000055,0.001873,NaN,...,0.000493,0.000535,0.001061,3.740015e-09,0.000204,0.000361,0.004578,0.000976,0.000995,0.004112


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                       \
Admin2               Autauga  Baldwin       Barbour          Bibb   
2021-06-21      3.111853e-07  0.00020  5.179830e-10  2.260244e-16   
2021-06-22      1.555926e-07  0.00010  2.589915e-10  1.130122e-16   
2021-06-23      7.779631e-08  0.00005  8.474576e-03  5.650609e-17   

Province_State                                                                \
Admin2                Blount       Bullock        Butler   Calhoun  Chambers   
2021-06-21      2.668582e-14  8.873674e-14  2.196224e-10  0.000385  0.000123   
2021-06-22      1.334291e-14  4.436837e-14  1.098112e-10  0.000192  0.000062   
2021-06-23      6.671455e-15  2.218419e-14  5.490561e-11  0.003154  0.000031   

Province_State                ...       Wyoming                              \
Admin2              Cherokee  ...          Park        Platte      Sheridan   
2021-06-21      2.466558e-21  ...  1.392931e-29  5.653853e-46  1.052029e-31   
2021-06-22      1.233279e-21  ...  6.964654e-30  2.826927e-46  5.260143e-32   
2021-06-23      6.166395e-22  ...  3.482327e-30  4.545455e-02  2.630071e-32   

Province_State                                                              \
Admin2              Sublette Sweetwater     Teton         Uinta Unassigned   
2021-06-21      4.621656e-52    0.00020  0.000027  1.480297e-16   0.752941   
2021-06-22      2.310828e-52    0.00010  0.000014  7.401487e-17   0.752941   
2021-06-23      1.155414e-52    0.00005  0.000007  3.700743e-17   0.752941   

Province_State                          
Admin2              Washakie    Weston  
2021-06-21      2.871513e-44  0.000024  
2021-06-22      1.435756e-44  0.000012  
2021-06-23      7.178782e-45  0.000006  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
